<a href="https://colab.research.google.com/github/PATELOM925/Automatic-Paper-Checker/blob/main/2nd_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (12.4 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120879 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [8]:
!pip install pytesseract


In [9]:
pip install pytesseract opencv-python pillow spacy

In [10]:
!python -m spacy download en_core_web_sm


2023-10-06 06:09:44.337646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from google.colab import files

# Use the files.upload() method to upload the image
uploaded = files.upload()


In [25]:
import cv2
import pytesseract
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from PIL import Image
import numpy as np

# Load spaCy model (you can choose a different language model if needed)
nlp = spacy.load("en_core_web_sm")

# Initialize NLTK sentiment analyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Preprocess image
def preprocess_image(image_path):
    # Reading image
    image = cv2.imread(image_path, 0)  # Read as grayscale

    # Adaptive thresholding
    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours
    min_contour_area = 100
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Drawing contours on the original image
    segmented_image = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), filtered_contours, -1, (0, 255, 0), 2)

    return segmented_image

# Function to perform OCR on the input image
def perform_ocr(image_path):
    segmented_image = preprocess_image(image_path)
    recognized_text = pytesseract.image_to_string(segmented_image, config='--psm 6')
    return recognized_text

# Function to process the recognized text using NLP techniques
def process_text(recognized_text):
    # Tokenization using spaCy
    doc = nlp(recognized_text)

    # Named Entity Recognition (NER) using spaCy
    for ent in doc.ents:
        print(f"Entity: {ent.text}, Label: {ent.label_}")

    # Sentiment analysis using NLTK
    sentiment_scores = sia.polarity_scores(recognized_text)
    sentiment = "Positive" if sentiment_scores["compound"] > 0 else "Negative" if sentiment_scores["compound"] < 0 else "Neutral"

    print("Recognized Text: ", recognized_text)
    print("Sentiment: ", sentiment)

# Image path
image_path = input("Enter the path to the image: ")

# Calling OCR function and providing image path
recognized_text = perform_ocr(image_path)

# Processing the recognized text using NLP techniques
process_text(recognized_text)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Enter the path to the image: output_processed.png
Entity: Aor Fe Beane, Label: PERSON
Entity: PE TA, Label: ORG
Entity: 0H, Label: CARDINAL
Recognized Text:  jt Bote
| polite Aor Fe Beane a
Fe Melictontey PP gone PE TA toptte
eee sod SE sale 0H
fe eee

Sentiment:  Positive
